In [ ]:
from fastai.vision.all import *

In [ ]:
pd.options.display.max_columns = 100

In [ ]:
datapath = Path("../rsna_data/")
embspath = Path(datapath/"cnn_embs")
train_df = pd.read_csv(datapath/'train.csv')

In [ ]:
list(embspath.ls())

### Load Embeddings & Preds

In [ ]:
fold = 4
folddir = embspath/f'full_512_ALL_FROM_FOLD{fold}'; list(folddir.ls())

In [ ]:
embeddings = torch.cat([torch.load(folddir/f"xresnet34_embeddings_{o}.pth") for o in ["part0", "part1", "part2", "finalpart"]])

In [ ]:
preds = torch.load(folddir/"preds.pth")

In [ ]:
files = torch.load(folddir/"files.pth")

In [ ]:
embeddings.shape, preds.shape, len(files)

In [ ]:
# add zero for padded input idx
input_pad_idx = len(embeddings)
embeddings = torch.cat([embeddings, torch.zeros_like(embeddings[:1])])

In [ ]:
embeddings[input_pad_idx], embeddings.shape, input_pad_idx

In [ ]:
preds[:,1]

### Metadata Features

In [ ]:
metadata_path = datapath/'metadata'
metadata_files = get_files(metadata_path, extensions=".csv")
metadf = pd.concat([pd.read_csv(o) for o in metadata_files]).reset_index(drop=True)

In [ ]:
metadf.shape

In [ ]:
len(metadf['StudyInstanceUID'].unique())

In [ ]:
def minmax_scaler(o): return (o - min(o))/(max(o) - min(o))

In [ ]:
scaled_pos = metadf.groupby('StudyInstanceUID')['ImagePositionPatient2'].apply(minmax_scaler)
metadf.loc[:,'scaled_position'] = scaled_pos.values

In [ ]:
meta_feat_cols = ['scaled_position']

In [ ]:
assert np.isnan(metadf[meta_feat_cols]).sum().sum() == 0

In [ ]:
mean_std = metadf[meta_feat_cols].agg(['mean', 'std']).T

In [ ]:
mean_std_dict = dict(zip(mean_std.index, mean_std.values.tolist())); mean_std_dict

In [ ]:
# standard scaler for training
for c in mean_std_dict: metadf[c] = (metadf[c] - mean_std_dict[c][0]) / mean_std_dict[c][1]

In [ ]:
meta_feats_dict = dict(zip(metadf['SOPInstanceUID'], metadf[meta_feat_cols].to_numpy()))

In [ ]:
len(meta_feats_dict)

### Fold Metadata

In [ ]:
meta_embeddings = []
for o in files:
    sopid = o.stem.split("_")[1]
    meta_embeddings.append(meta_feats_dict[sopid])
meta_embeddings = np.vstack(meta_embeddings)
meta_embeddings= tensor(meta_embeddings)

In [ ]:
use_preds = True
if use_preds:
    meta_embeddings = torch.cat([meta_embeddings, preds[:,1].view(-1,1).float()],1)

In [ ]:
meta_embeddings.shape, type(meta_embeddings)

In [ ]:
meta_embeddings = torch.cat([meta_embeddings, torch.zeros_like(meta_embeddings[:1])])

In [ ]:
embeddings.shape, meta_embeddings.shape

In [ ]:
combined_embeddings = torch.cat([embeddings, meta_embeddings], 1)

In [ ]:
combined_embeddings.shape

In [ ]:
combined_embeddings

### Data

In [ ]:
from fastai.text.all import *

In [ ]:
#         # positive, negative, indeterminate
#         self.pe_head = nn.Linear(dim//4, 3) # softmax
#         # rv / lv >=,  < 1 or neither
#         self.rv_lv_head = nn.Linear(dim//4, 3) # softmax
#         # l,r,c pe
#         self.pe_position_head = nn.Linear(dim//4, 3) # sigmoid
#         # chronic, ac-chr or neither
#         self.chronic_pe_head = nn.Linear(dim//4, 3) # softmax

In [ ]:
image_targets = L(['pe_present_on_image'])
exam_targets = L([
#           'positive_exam_for_pe'
            'negative_exam_for_pe',
            'indeterminate',

            'rv_lv_ratio_gte_1',
            'rv_lv_ratio_lt_1',
    # none

            'leftsided_pe',
            'rightsided_pe',
            'central_pe',

            'chronic_pe',
            'acute_and_chronic_pe',           
            # neither chronic or acute_and_chronic
          
    
    
#             'qa_motion',
#             'qa_contrast',
#             'flow_artifact',
#             'true_filling_defect_not_pe',
             ]); exam_targets

In [ ]:
targets_df = train_df[['StudyInstanceUID', 'SeriesInstanceUID', 'SOPInstanceUID']+image_targets+exam_targets]

In [ ]:
targets_dict = dict(zip(targets_df['SOPInstanceUID'].values, targets_df[image_targets+exam_targets].values))

In [ ]:
len(targets_dict)

In [ ]:
files_dict = defaultdict(list)
for i,o in enumerate(files):
    slice_no, sopid = o.stem.split("_")
    sid = o.parent.name
    slice_no = int(slice_no)        
    files_dict[sid].append({"slice_no":slice_no, "embs_idx":i, "img_y":targets_dict[sopid][0], "exam_y":targets_dict[sopid][1:]})

In [ ]:
all_pids = list(files_dict.keys())

In [ ]:
len(all_pids)

In [ ]:
exam_arr = list(files_dict[all_pids[0]][0]['exam_y'])

In [ ]:
exam_arr

In [ ]:
def get_x(pid, files_dict):
    o = files_dict[pid]    
    l = sorted(o, key=lambda x: x['slice_no']) 
    return tensor([o['embs_idx'] for o in l])

def get_img_y(pid, files_dict):
    o = files_dict[pid]    
    l = sorted(o, key=lambda x: x['slice_no']) 
    img_y = [o['img_y'] for o in l]
    img_y = tensor(img_y).float()
    return img_y

def get_exam_y(pid, files_dict):
    """ 4 prediction heads
    'POSITIVE','negative_exam_for_pe','indeterminate' - Sotfmax
    
    'rv_lv_ratio_gte_1','rv_lv_ratio_lt_1', 'NEITHER' - Softmax
    
    'leftsided_pe','rightsided_pe','central_pe' - Sigmoid
    
    'chronic_pe','acute_and_chronic_pe','NEITHER' - Sotfmax
    """
    d = files_dict[pid][0]
    exam_arr = list(d['exam_y'])

    # add POSITIVE
    is_pos = 0 if max(exam_arr[:2]) == 1 else 0
    exam_arr = [is_pos] + exam_arr
    
    # add NEITHER for rv/lv
    rvlv_neither = 1 if not is_pos else 0
    exam_arr = exam_arr[:5] + [rvlv_neither] + exam_arr[5:]
    
    # add NEITHER for chronic or acute
    chro_acc_neither = 1 if not is_pos else 0
    exam_arr = exam_arr + [chro_acc_neither]

    exam_y = tensor(exam_arr).float()
    return exam_y
    
# before_batch: after collecting samples before collating
targ_pad_idx = 666
def SequenceBlock():       return  TransformBlock(type_tfms=[partial(get_x, files_dict=files_dict)], 
                                                  dl_type=SortedDL,
                                                  dls_kwargs={'before_batch':
                                                               [partial(pad_input, pad_idx=input_pad_idx),
                                                                partial(pad_input, pad_idx=targ_pad_idx, pad_fields=1)]})
def SequenceTargetBlock(): return TransformBlock(type_tfms=[partial(get_img_y, files_dict=files_dict)])
def TargetBlock():         return TransformBlock(type_tfms=[partial(get_exam_y, files_dict=files_dict)])

In [ ]:
get_img_y(all_pids[0], files_dict)

In [ ]:
get_exam_y(all_pids[0], files_dict)

In [ ]:
files_dict[all_pids[0]][0]

In [ ]:
# normalized_embeddings = F.normalize(combined_embeddings, dim=0)
# normalized_embeddings.isnan().sum()
# normalized_embeddings

In [ ]:
assert combined_embeddings.isnan().sum().item() == 0

### Model

In [ ]:
device = default_device(1); device

In [ ]:
class AWD_LSTM(Module):
    "AWD-LSTM inspired by https://arxiv.org/abs/1708.02182"
    initrange=0.1

    def __init__(self, emb_sz,n_hid, n_layers, hidden_p=0.2, input_p=0.6, weight_p=0.5, bidir=False):
        store_attr('emb_sz,n_hid,n_layers')
        self.bs = 1
        self.n_dir = 2 if bidir else 1
        
        self.rnns = nn.ModuleList([self._one_rnn(emb_sz if l == 0 else n_hid, (n_hid)//self.n_dir, bidir, weight_p, l) for l in range(n_layers)])

        self.input_dp = RNNDropout(input_p)
        self.hidden_dps = nn.ModuleList([RNNDropout(hidden_p) for l in range(n_layers)])
        self.reset()

    def forward(self, x, from_embeds=False):
        
        if from_embeds: inp = x
        else: inp = combined_embeddings[x].to(device)
        bs,sl = inp.shape[:2]
        if bs!=self.bs: self._change_hidden(bs)

        output = self.input_dp(inp)
        new_hidden = []
        for l, (rnn,hid_dp) in enumerate(zip(self.rnns, self.hidden_dps)):
            output, new_h = rnn(output, self.hidden[l])
            new_hidden.append(new_h)
            if l != self.n_layers - 1: output = hid_dp(output)
        self.hidden = to_detach(new_hidden, cpu=False, gather=False)
        return output

    def _change_hidden(self, bs):
        self.hidden = [self._change_one_hidden(l, bs) for l in range(self.n_layers)]
        self.bs = bs

    def _one_rnn(self, n_in, n_out, bidir, weight_p, l):
        "Return one of the inner rnn"
        rnn = nn.LSTM(n_in, n_out, 1, batch_first=True, bidirectional=bidir, bias=False)
        return WeightDropout(rnn, weight_p)

    def _one_hidden(self, l):
        "Return one hidden state"
        nh = (self.n_hid) // self.n_dir
        return (one_param(self).new_zeros(self.n_dir, self.bs, nh), one_param(self).new_zeros(self.n_dir, self.bs, nh))

    def _change_one_hidden(self, l, bs):
        if self.bs < bs:
            nh = (self.n_hid) // self.n_dir
            return tuple(torch.cat([h, h.new_zeros(self.n_dir, bs-self.bs, nh)], dim=1) for h in self.hidden[l])
        if self.bs > bs: return (self.hidden[l][0][:,:bs].contiguous(), self.hidden[l][1][:,:bs].contiguous())
        return self.hidden[l]

    def reset(self):
        "Reset the hidden states"
        [r.reset() for r in self.rnns if hasattr(r, 'reset')]
        self.hidden = [self._one_hidden(l) for l in range(self.n_layers)]

In [ ]:
lstm_width = 512
layers = [lstm_width * 3] + [lstm_width] + [12]

class MultiHeadedSoftmaxSequenceClassifier(Module):
    "dim: input sequence feature dim"
    def __init__(self, bptt=72, input_pad_idx=input_pad_idx, n_meta=1, dim=1024, nlayers=2, cls_ps=[0.4, 0.1], **awd_kwargs):
        
        store_attr('input_pad_idx')
        self.awd_lstm = AWD_LSTM(dim+n_meta, lstm_width, nlayers, bidir=True, **awd_kwargs)
#         self.awd_lstm = AWD_QRNN(dim+n_meta, 512, 2, bidir=True)
        self.encoder = SentenceEncoder(bptt=bptt, module=self.awd_lstm, pad_idx=input_pad_idx)
        
        # image level preds
        self.seq_head = LinearDecoder(1, lstm_width, bias=True)
 
        # exam level preds
        self.exam_head = PoolingLinearClassifier(layers, ps=cls_ps, bptt=bptt)
        
    
    def forward(self, x):
        out, mask = self.encoder(x) 
       
        # img level out
        seq_cls_out,_,_ = self.seq_head(out)
        seq_cls_out = seq_cls_out.squeeze(-1)
              
        # exam level out
        exam_out,_,_ = self.exam_head((out,mask))
        posneg_out, rvlv_out, lrc_out, chroacute_out = (exam_out[:,:3], 
                                                        exam_out[:,3:6], 
                                                        exam_out[:,6:9], 
                                                        exam_out[:,9:])

        return (seq_cls_out, posneg_out, rvlv_out, lrc_out, chroacute_out)

In [ ]:
class MultiLossSoftmax(Module):
    
    def __init__(self, targ_pad_idx=666):
        store_attr("targ_pad_idx")

    def forward(self, inp, yb0, yb1):

        seq_cls_out, posneg_out, rvlv_out, lrc_out, chroacute_out = inp

        #             'negative_exam_for_pe',
        #             'indeterminate',

        #             'rv_lv_ratio_gte_1',
        #             'rv_lv_ratio_lt_1',
        #     # none

        #             'leftsided_pe',
        #             'rightsided_pe',
        #             'central_pe',

        #             'chronic_pe',
        #             'acute_and_chronic_pe', 
        image_target_weight = 0.07361963
        exam_target_weights = tensor([0.0736196319, 0.09202453988, 
                                      0.2346625767, 0.0782208589,
                                      0.06257668712, 0.06257668712, 0.1877300613,
                                      0.1042944785, 0.1042944785]).to(yb1.device)
        # img loss
        mask = yb0 != self.targ_pad_idx 

        img_loss, qs = 0, 0        
        for _m,_y,_p in zip(mask, yb0, seq_cls_out):
            qi = _y[_m].mean()
            qs += image_target_weight*qi*sum(_m)
            img_loss += image_target_weight*qi*(F.binary_cross_entropy_with_logits(_p[_m], _y[_m], reduction='sum'))

        # exam loss
        # 'POSITIVE','negative_exam_for_pe','indeterminate'
        posneg_out = posneg_out.softmax(1)
        posneg_losses = F.binary_cross_entropy(posneg_out, yb1[:,:3], reduction='none')[:, 1:]
        tot_posneg_losses = (posneg_losses*(exam_target_weights[:2].unsqueeze(0))).sum()
        tot_posneg_wgts = len(posneg_losses)*(tensor(exam_target_weights[:2]).sum())

        # 'rv_lv_ratio_gte_1','rv_lv_ratio_lt_1', 'NEITHER'
        rvlv_out = rvlv_out.softmax(1)
        rvlv_losses = F.binary_cross_entropy(rvlv_out, yb1[:,3:6], reduction='none')[:, :2]
        tot_rvlv_losses = (rvlv_losses*(exam_target_weights[2:4].unsqueeze(0))).sum()
        tot_rvlv_wgts = len(rvlv_losses)*(tensor(exam_target_weights[2:4]).sum())

        # 'leftsided_pe','rightsided_pe','central_pe'
        lrc_losses = F.binary_cross_entropy_with_logits(lrc_out, yb1[:,6:9], reduction='none')
        tot_lrc_losses = (lrc_losses*(exam_target_weights[4:7].unsqueeze(0))).sum()
        tot_lrc_wgts = len(lrc_losses)*(tensor(exam_target_weights[6:9]).sum())

        # 'chronic_pe','acute_and_chronic_pe','NEITHER'
        chroacute_out = chroacute_out.softmax(1)
        chroacute_losses = F.binary_cross_entropy(chroacute_out, yb1[:,9:], reduction='none')[:, :2]
        tot_chroacute_losses = (chroacute_losses*(exam_target_weights[7:].unsqueeze(0))).sum()
        tot_chroacute_wgts = len(chroacute_losses)*(tensor(exam_target_weights[7:]).sum())

        tot_exam_loss = (tot_posneg_losses+tot_rvlv_losses+tot_lrc_losses+tot_chroacute_losses)
        tot_exam_wgts = (tot_posneg_wgts+tot_rvlv_wgts+tot_lrc_wgts+tot_chroacute_wgts)



        return (tot_exam_loss+img_loss)/(qs+tot_exam_wgts)

In [ ]:
class ImageLossSoftmax(Module):
    
    def __init__(self, targ_pad_idx=666):
        store_attr("targ_pad_idx")

    def forward(self, inp, yb0, yb1):

        seq_cls_out, posneg_out, rvlv_out, lrc_out, chroacute_out = inp

        #             'negative_exam_for_pe',
        #             'indeterminate',

        #             'rv_lv_ratio_gte_1',
        #             'rv_lv_ratio_lt_1',
        #     # none

        #             'leftsided_pe',
        #             'rightsided_pe',
        #             'central_pe',

        #             'chronic_pe',
        #             'acute_and_chronic_pe', 
        image_target_weight = 0.07361963
        exam_target_weights = tensor([0.0736196319, 0.09202453988, 
                                      0.2346625767, 0.0782208589,
                                      0.06257668712, 0.06257668712, 0.1877300613,
                                      0.1042944785, 0.1042944785]).to(yb1.device)

        # img loss
        mask = yb0 != self.targ_pad_idx 

        img_loss, qs = 0, 0        
        for _m,_y,_p in zip(mask, yb0, seq_cls_out):
            qi = _y[_m].mean()
            qs += image_target_weight*qi*sum(_m)
            img_loss += image_target_weight*qi*(F.binary_cross_entropy_with_logits(_p[_m], _y[_m], reduction='sum'))
        return (img_loss)/(qs)

In [ ]:
class ExamLossSoftmax(Module):
    
    def __init__(self, targ_pad_idx=666):
        store_attr("targ_pad_idx")

    def forward(self, inp, yb0, yb1):

        seq_cls_out, posneg_out, rvlv_out, lrc_out, chroacute_out = inp

        #             'negative_exam_for_pe',
        #             'indeterminate',

        #             'rv_lv_ratio_gte_1',
        #             'rv_lv_ratio_lt_1',
        #     # none

        #             'leftsided_pe',
        #             'rightsided_pe',
        #             'central_pe',

        #             'chronic_pe',
        #             'acute_and_chronic_pe', 
        image_target_weight = 0.07361963
        exam_target_weights = tensor([0.0736196319, 0.09202453988, 
                                      0.2346625767, 0.0782208589,
                                      0.06257668712, 0.06257668712, 0.1877300613,
                                      0.1042944785, 0.1042944785]).to(yb1.device)


        # exam loss
        # 'POSITIVE','negative_exam_for_pe','indeterminate'
        posneg_out = posneg_out.softmax(1)
        posneg_losses = F.binary_cross_entropy(posneg_out, yb1[:,:3], reduction='none')[:, 1:]
        tot_posneg_losses = (posneg_losses*(exam_target_weights[:2].unsqueeze(0))).sum()
        tot_posneg_wgts = len(posneg_losses)*(tensor(exam_target_weights[:2]).sum())

        # 'rv_lv_ratio_gte_1','rv_lv_ratio_lt_1', 'NEITHER'
        rvlv_out = rvlv_out.softmax(1)
        rvlv_losses = F.binary_cross_entropy(rvlv_out, yb1[:,3:6], reduction='none')[:, :2]
        tot_rvlv_losses = (rvlv_losses*(exam_target_weights[2:4].unsqueeze(0))).sum()
        tot_rvlv_wgts = len(rvlv_losses)*(tensor(exam_target_weights[2:4]).sum())

        # 'leftsided_pe','rightsided_pe','central_pe'
        lrc_losses = F.binary_cross_entropy_with_logits(lrc_out, yb1[:,6:9], reduction='none')
        tot_lrc_losses = (lrc_losses*(exam_target_weights[4:7].unsqueeze(0))).sum()
        tot_lrc_wgts = len(lrc_losses)*(tensor(exam_target_weights[6:9]).sum())

        # 'chronic_pe','acute_and_chronic_pe','NEITHER'
        chroacute_out = chroacute_out.softmax(1)
        chroacute_losses = F.binary_cross_entropy(chroacute_out, yb1[:,9:], reduction='none')[:, :2]
        tot_chroacute_losses = (chroacute_losses*(exam_target_weights[7:].unsqueeze(0))).sum()
        tot_chroacute_wgts = len(chroacute_losses)*(tensor(exam_target_weights[7:]).sum())

        tot_exam_loss = (tot_posneg_losses+tot_rvlv_losses+tot_lrc_losses+tot_chroacute_losses)
        tot_exam_wgts = (tot_posneg_wgts+tot_rvlv_wgts+tot_lrc_wgts+tot_chroacute_wgts)



        return (tot_exam_loss)/(tot_exam_wgts)

### Train

In [ ]:
do_cv = True
FOLD = fold

if do_cv: 
    valid_pids = pd.read_pickle(datapath/f'cv_pids/pids_fold{FOLD}.pkl')

In [ ]:
len(valid_pids)

In [ ]:
data = DataBlock(blocks=(SequenceBlock,SequenceTargetBlock,TargetBlock), 
                 n_inp=1, 
                 splitter=FuncSplitter(lambda o: True if o in valid_pids else False)
                )
dls = data.dataloaders(all_pids, bs=128)
model = SequentialRNN(MultiHeadedSoftmaxSequenceClassifier(bptt=256, n_meta=2, dim=1024, nlayers=2)) # dim = 1536 for effnet 1024 for xresnet
loss_func = MultiLossSoftmax()
learner = Learner(dls, model, loss_func=loss_func, metrics=[ImageLossSoftmax(), ExamLossSoftmax()],
                  cbs=[ModelResetter(),TerminateOnNaNCallback(), 
                      SaveModelCallback(fname=f"nometa_sequence_softmax_with_preds_fulldata_fold{fold}")])

In [ ]:
# learner.lr_find()

In [ ]:
learner.fit_flat_cos(20, lr=0.005)

### Get OOF Preds

In [ ]:
def get_pid(pid):
    return pid

def get_x(pid, files_dict):
    o = files_dict[pid]    
    l = sorted(o, key=lambda x: x['slice_no']) 
    return tensor([o['embs_idx'] for o in l])

def get_img_y(pid, files_dict):
    o = files_dict[pid]    
    l = sorted(o, key=lambda x: x['slice_no']) 
    img_y = [o['img_y'] for o in l]
    img_y = tensor(img_y).float()
    return img_y

def get_exam_y(pid, files_dict):
    """ 4 prediction heads
    'POSITIVE','negative_exam_for_pe','indeterminate' - Sotfmax
    
    'rv_lv_ratio_gte_1','rv_lv_ratio_lt_1', 'NEITHER' - Softmax
    
    'leftsided_pe','rightsided_pe','central_pe' - Sigmoid
    
    'chronic_pe','acute_and_chronic_pe','NEITHER' - Sotfmax
    """
    d = files_dict[pid][0]
    exam_arr = list(d['exam_y'])

    # add POSITIVE
    is_pos = 0 if max(exam_arr[:2]) == 1 else 0
    exam_arr = [is_pos] + exam_arr
    
    # add NEITHER for rv/lv
    rvlv_neither = 1 if not is_pos else 0
    exam_arr = exam_arr[:5] + [rvlv_neither] + exam_arr[5:]
    
    # add NEITHER for chronic or acute
    chro_acc_neither = 1 if not is_pos else 0
    exam_arr = exam_arr + [chro_acc_neither]

    exam_y = tensor(exam_arr).float()
    return exam_y
    
# before_batch: after collecting samples before collating
targ_pad_idx = 666
def SequenceBlock():       return  TransformBlock(type_tfms=[partial(get_x, files_dict=files_dict)], 
                                                  dl_type=SortedDL,
                                                  dls_kwargs={'before_batch':
                                                               [partial(pad_input, pad_idx=input_pad_idx),
                                                                partial(pad_input, pad_idx=targ_pad_idx, pad_fields=1)]})
def PidBlock(): return TransformBlock(type_tfms=[get_pid])
def SequenceTargetBlock(): return TransformBlock(type_tfms=[partial(get_img_y, files_dict=files_dict)])
def TargetBlock():         return TransformBlock(type_tfms=[partial(get_exam_y, files_dict=files_dict)])

In [ ]:
data = DataBlock(blocks=(SequenceBlock,SequenceTargetBlock,TargetBlock), 
                 n_inp=1, 
                 splitter=FuncSplitter(lambda o: True if o in valid_pids else False)
                )
dls = data.dataloaders(all_pids, bs=128)
model = SequentialRNN(MultiHeadedSoftmaxSequenceClassifier(bptt=256, n_meta=2, dim=1024, nlayers=2)) # dim = 1536 for effnet 1024 for xresnet
loss_func = MultiLossSoftmax()
learner = Learner(dls, model, loss_func=loss_func, metrics=[ImageLossSoftmax(), ExamLossSoftmax()],
                  cbs=[ModelResetter(),TerminateOnNaNCallback(), 
                      SaveModelCallback(fname=f"nometa_sequence_softmax_with_preds_fulldata_fold{fold}")])

In [ ]:
learner.load(f"nometa_sequence_softmax_with_preds_fulldata_fold{fold}");

In [ ]:
learner.model.eval().to(device);

In [ ]:
len(all_pids)

In [ ]:
test_dl = learner.dls.test_dl(all_pids, with_labels=True)

In [ ]:
seq_pids = []
seq_img_preds = []
seq_img_targs = []
seq_exam_preds = []
seq_exam_targs = []
with torch.no_grad():
    for xb,yb0,yb1,pids in progress_bar(test_dl):
        img_pred, exam_pred = to_detach(learner.model(xb))
        seq_img_preds.append(img_pred)
        seq_img_targs.append(yb0)
        seq_exam_preds.append(exam_pred)
        seq_exam_targs.append(yb1)
        seq_pids.append(pids)

In [ ]:
stacking_datapath = datapath/'final_lstm_stacking'
if not stacking_datapath.exists(): stacking_datapath.mkdir()

In [ ]:
subfolder = f"xresnet_softmax_FOLD{FOLD}"
stacking_folder = stacking_datapath/subfolder
if not stacking_folder.exists(): stacking_folder.mkdir()

In [ ]:
len(seq_pids), len(seq_img_preds), len(seq_img_targs), len(seq_exam_preds), len(seq_exam_targs), len(valid_pids)

In [ ]:
torch.save(seq_pids, stacking_folder/'seq_pids.pth')
torch.save(seq_img_preds, stacking_folder/'seq_img_preds.pth')
torch.save(seq_img_targs, stacking_folder/'seq_img_targs.pth')
torch.save(seq_exam_preds, stacking_folder/'seq_exam_preds.pth')
torch.save(seq_exam_targs, stacking_folder/'seq_exam_targs.pth')
torch.save(valid_pids, stacking_folder/'valid_pids.pth')